In [1]:
from utils import *
import re
from pprint import pprint
from nltk.corpus import stopwords

all_data = get_data_from_db()

num_remover = NumRemover()
word_remover = WordRemover(stopwords.words('portuguese'))
all_data = num_remover.fit_transform(all_data)
all_data = word_remover.fit_transform(all_data)

neg_data = all_data[all_data["labels"] == "NG"]
neu_data = all_data[all_data["labels"] == "NE"]
pos_data = all_data[all_data["labels"] == "PO"]

ratio = 0.7
neg_train = neg_data.iloc[0:round(ratio*neg_data.shape[0]), :]
neu_train = neu_data.iloc[0:round(ratio*neu_data.shape[0]), :]
pos_train = pos_data.iloc[0:round(ratio*pos_data.shape[0]), :]

neg_test = neg_data.iloc[round(ratio*neg_data.shape[0]): , :]
neu_test = neu_data.iloc[round(ratio*neu_data.shape[0]): , :]
pos_test = pos_data.iloc[round(ratio*neg_data.shape[0]): , :]

stopwords_pt = stopwords.words("portuguese")

# Vocabulários

In [4]:
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer

vocab_neg = set(' '.join(neg_train["texts"].tolist()).split())
vocab_neu = set(' '.join(neu_train["texts"].tolist()).split())
vocab_pos = set(' '.join(pos_train["texts"].tolist()).split())

print("Dimensão vocabulario negativo: " + str(len(vocab_neg)))
print("Dimensão vocabulario neutro: " + str(len(vocab_neu)))
print("Dimensão vocabulario positivo: " + str(len(vocab_pos)))

Dimensão vocabulario negativo: 2687
Dimensão vocabulario neutro: 2145
Dimensão vocabulario positivo: 2326


In [5]:
# Verificar interseção dos vocabulários
intersect = vocab_neg.intersection(vocab_pos.intersection(vocab_neu))
vocab_neg_excl = vocab_neg.difference(intersect)
vocab_neu_excl = vocab_neu.difference(intersect)
vocab_pos_excl = vocab_pos.difference(intersect)

lexicon = list(vocab_neg.union(vocab_pos))

In [8]:
from sklearn.naive_bayes  import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

features = FeatureUnion([
#                     ("bigram", CountVectorizer(ngram_range=(2,2), stop_words= stopwords_pt, binary= True)),
                    ("lexicon_vector", CountVectorizer(vocabulary= lexicon, binary = True))
                    ])

folds = 10;
print("MultinomialNB")
run_cross_validation(all_data, features, MultinomialNB(), n_folds= folds)

print("\n\nSVM")
run_cross_validation(all_data, features, SVC(C=316), n_folds=folds)

print("\nMaxent")
run_cross_validation(all_data, features, LogisticRegressionCV(fit_intercept=False, penalty= 'l2', dual= False), n_folds=folds)

MultinomialNB
Cross Validation:
Fold  0  - Acuracia:  0.537735849057
Fold  1  - Acuracia:  0.490384615385
Fold  2  - Acuracia:  0.528846153846
Fold  3  - Acuracia:  0.5
Fold  4  - Acuracia:  0.509615384615
Fold  5  - Acuracia:  0.576923076923
Fold  6  - Acuracia:  0.596153846154
Fold  7  - Acuracia:  0.586538461538
Fold  8  - Acuracia:  0.586538461538
Fold  9  - Acuracia:  0.548076923077

Accuracia media:  0.546081277213
Desvio padrão:  0.0369568506763


SVM
Cross Validation:
Fold  0  - Acuracia:  0.584905660377
Fold  1  - Acuracia:  0.528846153846
Fold  2  - Acuracia:  0.442307692308
Fold  3  - Acuracia:  0.480769230769
Fold  4  - Acuracia:  0.576923076923
Fold  5  - Acuracia:  0.548076923077
Fold  6  - Acuracia:  0.576923076923
Fold  7  - Acuracia:  0.557692307692
Fold  8  - Acuracia:  0.586538461538
Fold  9  - Acuracia:  0.528846153846

Accuracia media:  0.54118287373
Desvio padrão:  0.0453587081655

Maxent
Cross Validation:
Fold  0  - Acuracia:  0.556603773585
Fold  1  - Acuracia: 

In [14]:
# Avaliando a incidência dos bigramas
cv = CountVectorizer(ngram_range=(1,3))

neg_counts = cv.fit_transform(neg_train["texts"])
neg_bigrams = cv.vocabulary_

pos_counts = cv.fit_transform(pos_train["texts"])
pos_bigrams = cv.vocabulary_

neu_counts = cv.fit_transform(neu_train["texts"])
neu_bigrams = cv.vocabulary_

spo = np.sum(pos_counts.toarray(), axis=0)
sng = np.sum(neg_counts.toarray(), axis=0)
sne = np.sum(neu_counts.toarray(), axis=0)

pos_bigrams  = {bigram: spo[index] for bigram, index in pos_bigrams.items()}
neg_bigrams  = {bigram: sng[index] for bigram, index in neg_bigrams.items()}
neu_bigrams  = {bigram: sne[index] for bigram, index in neu_bigrams.items()}

pos_excl_bigrams = {key:value for key, value in pos_bigrams.items() if key not in neg_bigrams and key not in neu_bigrams}
neg_excl_bigrams = {key:value for key, value in neg_bigrams.items() if key not in pos_bigrams and key not in neu_bigrams}
neu_excl_bigrams = {key:value for key, value in neu_bigrams.items() if key not in pos_bigrams and key not in neg_bigrams}

In [17]:
from pandas import Series
s = Series(pos_excl_bigrams)
s.nlargest(15)

educação                    20
saúde                       10
saúde educação               6
américa                      5
américa latina               5
cada vez                     5
educação corrupção           5
investir                     5
latina                       5
punidos                      5
quatro anos                  5
saúde educação corrupção     5
cidade                       4
drogas                       4
educação saúde               4
dtype: int64

In [18]:
from sklearn.naive_bayes  import MultinomialNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

lexicon_bigrams = list(pos_excl_bigrams.keys()) + list(neg_excl_bigrams.keys())

features = FeatureUnion([
#                     ("bigram", CountVectorizer(ngram_range=(1,1), stop_words= stopwords_pt, vocabulary  = lexicon)),
                    ("lexicon_vector", CountVectorizer(ngram_range=(1,2), stop_words=stopwords_pt, vocabulary= lexicon_bigrams))
                    ])

folds = 10;
print("MultinomialNB")
run_cross_validation(all_data, features, MultinomialNB(), n_folds= folds)

print("\n\nSVM")
run_cross_validation(all_data, features, SVC(C=316), n_folds=folds)

print("\nMaxent")
run_cross_validation(all_data, features, LogisticRegressionCV(fit_intercept=False, penalty= 'l2', dual= False), n_folds=folds)

MultinomialNB
Cross Validation:
Fold  0  - Acuracia:  0.283018867925
Fold  1  - Acuracia:  0.163461538462
Fold  2  - Acuracia:  0.278846153846
Fold  3  - Acuracia:  0.153846153846
Fold  4  - Acuracia:  0.298076923077
Fold  5  - Acuracia:  0.298076923077
Fold  6  - Acuracia:  0.269230769231
Fold  7  - Acuracia:  0.230769230769
Fold  8  - Acuracia:  0.298076923077
Fold  9  - Acuracia:  0.163461538462

Accuracia media:  0.243686502177
Desvio padrão:  0.0578108407068


SVM
Cross Validation:
Fold  0  - Acuracia:  0.405660377358
Fold  1  - Acuracia:  0.625
Fold  2  - Acuracia:  0.394230769231
Fold  3  - Acuracia:  0.605769230769
Fold  4  - Acuracia:  0.644230769231
Fold  5  - Acuracia:  0.625
Fold  6  - Acuracia:  0.673076923077
Fold  7  - Acuracia:  0.615384615385
Fold  8  - Acuracia:  0.384615384615
Fold  9  - Acuracia:  0.605769230769

Accuracia media:  0.557873730044
Desvio padrão:  0.108444927807

Maxent
Cross Validation:
Fold  0  - Acuracia:  0.707547169811
Fold  1  - Acuracia:  0.7307